In [1]:
# !pip install pysimplegui
#pip install --force-reinstall -v "pyabf==2.3.8" #delite and reinstall the certain version
#pip install -Iv MySQL_python==1.2.2 # install the certain version
#!pip install matplotx==0.0.0

#import abf_explorer
#!pip install --upgrade numpy
import pyabf

import numpy as np

import matplotlib.pyplot as plt
#import matplotx

import statistics

import PySimpleGUI as sg

import pandas as pd

import os
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d

In [3]:
# GUI to choose folder
tres = 0
folder_path = False
while True:
    event, values = sg.Window('Choose your folder', [[sg.Text('Folder')], [sg.Input(), sg.FolderBrowse()], [sg.OK(), sg.Cancel()] ]).read(close=True)
    if event == 'Cancel':
        break
    try:
        folder_path = values[0]
        break
    except:
        print('Incorrect input')

In [4]:
num_points = 40 

#define the area of interest for the cell in 1st and 2nd devices 
def TimePeriodBorders (my_file_abf):
    sample_rate = int(my_file_abf.dataRate)
    if sample_rate == 20000: 
        # Time for PreStim and Stim for Channel 0 
        my_start10 = 0 
        my_end10 = 2800    
        my_start11 = 4000 
        my_end11 = 12800 
        # Time for PreStim and Stim for Channel 2
        my_start20 = 28000 
        my_end20 = 30800 
        my_start21 = 32000 
        my_end21 = 40800 
    elif sample_rate == 10000:
        # Time for PreStim and Stim for Channel 0
        my_start10 = 0 
        my_end10 = 1400     
        my_start11 = 2000 
        my_end11 = 6400 
        # Time for PreStim and Stim for Channel 2
        my_start20 = 14000
        my_end20 = 15400
        my_start21 = 16000
        my_end21 = 20000 
    else:
        print("Check the Sampling rate!")
        return ([],[],[],[],[],[],[],[])
        
    return (my_start10, my_end10, 
            my_start11, my_end11, 
            my_start20, my_start21, 
            my_start21, my_end21)



#find the last sweep with no APs    
def sweep_AP (my_file_abf, search_ch, start, stop):
    #choose the sweep with at least 1 spikes
    for sweep_num in range(len(my_file_abf.sweepList)):
        my_file_abf.setSweep(sweep_num, channel = search_ch)
        peaks = find_peaks(my_file_abf.sweepY[start:stop], height=0)
        #print(peaks, 'My peaks')
        if len(peaks[0])>=1:
            sweep = sweep_num-1 #sweep is detected
            break
        elif len(peaks[0])==0 and sweep_num==len(my_file_abf.sweepList)-1:
            #print('Sweep is not detected!')
            sweep = len(my_file_abf.sweepList)
    #print(sweep)
    return sweep



#relative values of I and V and Rin
def IV_small (my_file_abf, my_sweep_list,
                 voltage_channel, current_channel,
                 pre_start, pre_end, 
                 post_start, post_end, 
                 currents_delta_list = [], voltages_delta_list = [], voltage_range_ind_list = [],
                 Rin_relative_list = [], Rin_relative_norm_list = [], currents_range_list = [], voltages_range_list = []):
    
        voltages_dur_stim_list = []
    
        for my_sweepNumber in range(my_sweep_list):
            
            my_file_abf.setSweep(sweepNumber = my_sweepNumber, channel = voltage_channel)
            if voltage_channel==0:
                voltage_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                voltage_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
            elif voltage_channel==2:
                voltage_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                voltage_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
                
            voltage_delta = voltage_dur_stim - voltage_pre_stim
            
            my_file_abf.setSweep(sweepNumber = my_sweepNumber, channel = current_channel)
            if current_channel==1:
                current_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                current_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
            elif current_channel==3:
                current_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                current_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
                
            current_delta = current_dur_stim - current_pre_stim
            
            
            if abs(current_delta)<5:                
                voltages_dur_stim_list.append(voltage_dur_stim)
                currents_delta_list.append(current_delta) 
            
        return voltages_dur_stim_list, currents_delta_list

In [20]:
my_voltage_channel, my_current_channel = 0, 1
file_names_list = []
my_Rin_range = []
my_Rin_property = []
voltages = []
currents = []

#folder_path = 'C:/Users/user/Documents/IMBIT/05.10.2023'
file_list = os.listdir(folder_path)
#print(file_list) # List of all file names

col1 = 'voltage'
col2 = 'currents'


for filename in file_list:
    print(filename)
    file_names_list.append(filename)
    file_path = os.path.join(folder_path, filename)
    my_file = pyabf.ABF(file_path)
    
    time_points = TimePeriodBorders(my_file)
    last_no_AP_sweep = sweep_AP(my_file, 0, time_points[0], time_points[3])
    my_relative_values = IV_small(my_file, last_no_AP_sweep, my_voltage_channel, my_current_channel, time_points[0], time_points[1], time_points[2], time_points[3], [], [], [], [], [], [], [])
    
    if my_relative_values==([], []):
        continue

    #print(my_relative_values[1][0])
   
    voltages.append(my_relative_values[0][0])
    currents.append(my_relative_values[1][0])
    
    #print(voltages)
    #print(currents)



my_data = pd.DataFrame({col1:voltages,
                            col2:currents})


my_data.to_excel(r'C:\Users\user\Documents\IMBIT\Master\Rin relative\IVsmallch0.xlsx'.format(filename) , sheet_name='All data', index=False)

#print('Done!')

2023_07_13_0000.abf
2023_07_13_0004.abf
2023_07_17_0013.abf
2023_07_20_0004.abf
2023_07_24_0003.abf
2023_07_24_0014.abf
2023_07_27_0001.abf
2023_07_27_0015.abf
2023_07_27_0016.abf
2023_07_27_0020.abf
2023_08_03_0018.abf
2023_08_08_0000.abf
2023_08_08_0001.abf
2023_08_08_0009.abf
2023_08_08_0031.abf
2023_08_08_0062.abf
2023_08_08_0055.abf
2023_08_08_0056.abf
2023_08_08_0057.abf
2023_08_08_0058.abf
2023_08_08_0059.abf
2023_08_08_0063.abf
2023_08_10_0000.abf
2023_08_10_0003.abf
2023_08_10_0004.abf
2023_08_10_0042.abf
2023_08_10_0043.abf
2023_08_10_0044.abf
2023_08_10_0046.abf
2023_08_15_0000.abf
2023_08_15_0002.abf
2023_08_15_0038.abf
2023_08_15_0039.abf
2023_08_17_0002.abf
2023_08_17_0003.abf
2023_08_17_0052.abf
2023_08_17_0039.abf
2023_08_17_0053.abf
2023_08_21_0006.abf
2023_08_21_0000.abf
2023_08_21_0003.abf
2023_08_21_0004.abf
2023_08_21_0005.abf
2023_08_21_0014.abf
2023_08_24_0012.abf
2023_08_24_0000.abf
2023_08_24_0005.abf
2023_08_24_0006.abf
2023_08_24_0007.abf
2023_08_24_0009.abf
